In which I attempt to control multiple robots simultaneously!


In [1]:
# Foundational Sphero API, forked from Simon's Library
import sphero

In [2]:
# Tracking code
import cv2
# SpheroNav base classes
from tracker.trackingfilter import FilterSpheroBlueCover, FilterSpheroYellowCover, FilterGlow


from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker
from util import Vector2D

In [3]:
import SpheroTeam

# If project is rearchitected, rethink how to make functions accessible
# Look into python module / api design
from SpheroTeam.teamutil import readJsonFile, normalize_angle, angle_between_points
from SpheroTeam.navigation import calibrate_bot_direction, get_bot_position, get_team_offsets

from SpheroTeam.formations import roll_polygon, roll_push
from SpheroTeam.director import bot_go_to_point

In [4]:
# General Python Imports
import time
import os
import logging

#### SpheroTeam Setup


For more details about what each line does, refer to the previous notebook (6)


In [5]:
# This is a list of the colored "initial names" of the spheros you want to work with in this session
ACTIVEBOTS = ['RPB', 'RWR', 'ORG']

In [6]:
PROJ_ROOT = os.getcwd()
CONFIG = os.path.join(PROJ_ROOT, "config.json")
config = readJsonFile(CONFIG)
TEAM_COLORS = config['teamColors']


manager = sphero.SpheroManager()

# Initialize Sphero manager using options provided in the config file
manager= SpheroTeam.initialize.load_sphero_roster(manager, CONFIG)

# Connect to robots
bots = SpheroTeam.initialize.connect_sphero_team(manager, ACTIVEBOTS)

Sphero-RPB Try #0
IOError Occurred. Trying to connect again.
Sphero-RPB Try #1
Connected!
Sphero-RWR Try #0
Connected!
Sphero-ORG Try #0
Connected!


In [7]:
# Check power, robots shut down after dropping below 7 volts
SpheroTeam.print_team_status(bots)

Sphero-RPB:Battery OK | 8.03 V
NOTE! Removes wrong byte in start of header! Byte:  13
Sphero-RWR:Battery OK | 7.96 V
NOTE! Removes wrong byte in start of header! Byte:  13
Sphero-ORG:Battery OK | 8.05 V


In [8]:
TEAM_COLORS = config['teamColors']
TEAM_COLORS

[[255, 0, 0],
 [0, 255, 0],
 [0, 0, 255],
 [255, 0, 255],
 [255, 255, 0],
 [255, 133, 0],
 [255, 192, 203]]

#### Camera Tracker Setup



In which I try custom filters for yellow, red, green, and orange, find that the camera has trouble seeing ANY of them besides blue


In [9]:
tracker = ColorTracker(config['cameraID'])

In [10]:
imageX, imageY = tracker.image_size
print "Image: {} x {} pixels".format(imageX, imageY)

Image: 640 x 480 pixels


In [11]:
traceable_color = [0, 0, 255]  # RGB Blue
traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

In [12]:
# make sure everyone is off before calibrating.
for bot in bots:
    bot.set_rgb(0,0,0)

In [19]:
# Blue seems to be the most reliable color to track with.
# Be suspicious if any of the offsets are 0!!!
offsets = get_team_offsets(bots, traceable_blue, traceable_color, tracker)

Start (270,236)
End   (231,248)
Angle 162.897271031
'q' to quit, else calibrate next robot a
Start (352,270)
End   (422,285)
Angle 12.094757077
'q' to quit, else calibrate next robot a
Start (358,216)
End   (321,136)
Angle 245.179458665
'q' to quit, else calibrate next robot a


In [7]:
?roll_polygon

In [17]:
# Roll in a square to show it's working
# can roll in single direction using roll-push.
roll_polygon(bots, 1, offsets, 180, speed=120, TIMEOUT=1000)

Dispatch Time 0.161999940872


In [20]:
roll_polygon(bots, 1, offsets, 180, speed=50, TIMEOUT=1000)

Dispatch Time 0.213999986649


### Finally: Group sending of robots to points




One problem: when robot rolls off the viewing field. Maybe when a robot cannot be seen, it should roll in the opposite of the last angle it rolled in.


In [15]:
?get_bot_position()

In [ ]:
print imageX
print imageY

In [23]:
def team_go_to_points(bots, targets, offsets, traceable_object, traceable_color,
                      MAX_X=imageX, MAX_Y=imageY, TIMEOUT=200, stopRadius=25):
    '''
        Send each bot in the team to its designated point. 
        Bots drive in a straight line to
        each point.
        
        Alternate function would send each robot along paths (sequences of points)
        Perhaps each point might have its own Kp or timeout or speed.
    '''
    
    for bot in bots:
        bot.set_rgb(0,0,0)
    
    for i, bot in enumerate(bots):
        print("Bot {}".format(bot.bt_name))
        bot.set_rgb(traceable_color[0], traceable_color[1], traceable_color[2])
        time.sleep(2.5)
        
        targetX, targetY = targets[i]
        bot_go_to_point(bot, offsets[i], targetX, targetY,          
                        traceable_object, traceable_color, MAX_X, MAX_Y, tracker, 
                        TIMEOUT=250, stopRadius=stopRadius)  
        
        bot.set_rgb(0,0,0)
        

In [24]:
def team_go_to_paths(bots, paths, offsets, traceable_object, traceable_color,
                      MAX_X=imageX, MAX_Y=imageY, TIMEOUT=200, stopRadius=25):
    '''
        Send each bot in the team some points!
    '''
    
    for bot in bots:
        bot.set_rgb(0,0,0)
    
    for i, bot in enumerate(bots):
        print("Bot {}".format(bot.bt_name))
        bot.set_rgb(traceable_color[0], traceable_color[1], traceable_color[2])
        time.sleep(2)
        

        for target in paths[i]:
            targetX, targetY = target
            bot_go_to_point(bot, offsets[i], targetX, targetY,          
                traceable_object, traceable_color, MAX_X, MAX_Y, tracker, TIMEOUT=250,
                           stopRadius=stopRadius)  
        
        # Turn robot off when done
        bot.set_rgb(0,0,0) 

In [26]:
# Every robot, after some pushing activity, will use this path to skirt the perimeter
recoveryPaths = [
    [
        [320, 400],  # 
        [100, 245]
    ],
    
    [
        [320, 100],
        [100, 235]
    ],
    
    [
        [320, 400],  # 
        [100, 250]
    ],
    
    [
        [320, 100],
        [100, 230]
    ],
    
    [
        [320, 400],  # 
        [100, 255]
    ],
    [
        
        [320, 100],
        [100, 225]
    ],
    [
        [320, 100],
        [100, 225]
    ]
]

In [27]:
# Send robots to a cluster around 300, 150
targets = [ [300, 150 + i*5] for i in range(len(bots))] # space out the robots along x = 100

team_go_to_points(bots, targets, offsets, traceable_blue, traceable_color, stopRadius=15, )

Bot Sphero-RPB
Get from 311,102 to 300,150| Distance 49.244289009 / 102.907408671
Stopped at 324,131, with dist 30.61045573
Bot Sphero-RWR
Get from 301,117 to 300,155| Distance 38.0131556175 / 91.5074357588
Stopped at 302,148, with dist 7.28010988928
Bot Sphero-ORG
Get from 287,138 to 300,160| Distance 25.5538646784 / 59.4207731275


Stopped at 248,130, with dist 60.0333240792


### Now, move them together




In [28]:
SpheroTeam.set_team_colors(bots, TEAM_COLORS)    

In [29]:
# Test a pentagon
roll_polygon(bots, 5, offsets, 180, speed=60, TIMEOUT=1200)

Dispatch Time 0.162999868393
Dispatch Time 0.181999921799
Dispatch Time 0.164999961853
Dispatch Time 0.197000026703
Dispatch Time 0.164000034332


In [61]:
# Bring a lost sphero back if it rolled away (consider making an arena wall in future)
# SpheroTeam.roll_sphero(bots[0], 40, 180, offsets[0])

In [ ]:
# Roll all team along angle 0 at a speed of 70 for 3 seconds
roll_push(bots, 0, 70, offsets, TIMEOUT=3000)

In [30]:
# Show how robots would recover from "overshooting" the target by going to the outside of the arena
#, and then returning to a home area.
team_go_to_paths(bots, recoveryPaths, offsets, traceable_blue, traceable_color)

Bot Sphero-RPB
Get from 248,130 to 320,400| Distance 279.435144533 / 75.0685828219
Stopped at 358,408, with dist 38.8329756779
Get from 355,387 to 100,245| Distance 291.871547089 / -150.888195474
Stopped at 135,253, with dist 35.902646142
Bot Sphero-RWR
Get from 134,236 to 320,100| Distance 230.417013261 / -36.1736290458
Stopped at 319,82, with dist 18.0277563773
Get from 317,87 to 100,235| Distance 262.665186121 / 145.70493089
Stopped at 103,219, with dist 16.2788205961
Bot Sphero-ORG
Get from 90,237 to 320,400| Distance 281.902465402 / 35.3250346381
Stopped at 302,417, with dist 24.7588368063
Get from 326,432 to 100,250| Distance 290.172362571 / -141.155168343
Stopped at 92,232, with dist 19.6977156036


Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 422, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 306, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Pytho

Challenges: 

- Path planning is not the same thing as simply choosing to arrive at a point.
    - Need to come up with algorithm to avoid obstacles, knowing that you may miss goal points by 10-20 centimeters.
    
- Tuning the right KP parameter is tricky
- Coming up with recovery paths is tricky

- How to stop robots from bumping into each other


- When robot runs out of bounds, we can't see it... need to create a "fence" around the arena and that would fix it.

- Tracker can only reliably "see" 1 color at once, so workaround is only using that 1 color for tracking.

Alternate ideas that weren't implemented due to lack of time

- When swarm needs to converge on 1 point: have everyone drive at fixed speed

- drive for however long that distance takes in pixels

- Let everyone drive simultaneously by giving each robot a blind directive, and not using any feedback to tune its behavior (aka a "formation")


- Write a function that given a singular (x,y) point, generates a list of pixels that each robot would aim for, along with an "shape" and relevant parameters for how the robots should be arranged (aka box, line, circle, hexagon, etc)

- Write a function that given a singular target (x,y) point and a robot, define a list of points that the robot should visit before reaching the target points

